In [1]:
!pip install pandas | wc -l
!pip install tensorflow | wc -l
!pip install tensorflow-gpu | wc -l

5
43
43


In [2]:
pwd()

'\\\\wsl.localhost\\Ubuntu-20.04\\home\\mdc20\\repos\\personal\\project-fashion-finder'

In [18]:
CLASS_LIST = ["Tee","Tank","Dress","Shorts","Skirt","Jumpsuit","Sweater","Blazer","Striped","Cardigan","Blouse","Romper","Sweatpants","Jacket"]

In [66]:
import os
import numpy as np
import scipy
import PIL
PIL.Image.MAX_IMAGE_PIXELS = None
import tensorflow.keras as keras
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [67]:
from collections import defaultdict
import pandas as pd
datasetdir = 'C:\\Users\\mdc20\\Downloads\\img'
os.chdir(datasetdir)
def constructImageClassDataFrame(shape, min_class_occurence = 1, class_list = CLASS_LIST):
    # Args: 
    #   Shape: Image shape (2D)
    #   min_class_occurence: number of times class must occur in labeled dir in order to add it to the final sclass list
    
    
    sub_dirs = [d for d in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(datasetdir, d))]
    
    if class_list is None:
        classes = defaultdict(lambda: 0)

        for sub_dir in sub_dirs:
            labels = sub_dir.split('_')
            for label in labels:
                classes[label] += 1
        top_k_classes = [cls for cls in classes if classes[cls] >= min_class_occurence]
        label_classes = top_k_classes    
    else:
        classes = class_list
        label_classes = classes

    
    # classes = list(classes)
    # print("Done calculating list of classes")

    # TODO: remove
    # top_k_classes = ['Graphic', 'Tee', 'Tank', 'Dress', 'Shorts', 'Print', 'Skirt', 'Blouse', 'Top', 'Leggings', 'Sweater', 'Abstract', 'Romper', 'Jumpsuit', 'Floral', 'Cardigan',  'Stripe', 'Boxy','Joggers', 'Striped',  'Tribal', 'Jacket', 'Jeans' ]
    # top_k_classes = ['Top', 'Blouse']

    arr = [[labeled_dir, file] for labeled_dir in sub_dirs for file in os.listdir(labeled_dir) ]
    print("Done parsing through directories")
    df = pd.DataFrame(data=arr, columns=["folder","filename"])



    df['filename'] = df['folder'] + '/' + df['filename']
    df['labels'] = df['folder'].apply(lambda x : [y for y in x.split('_') if y in label_classes] if len([y for y in x.split('_') if y in label_classes]) > 0 else None)

    print(np.shape(df))
    # index_names = df [df['labels'] == []].index
    # Drop Dataframe if none of its labels are present in the selected classes
    df = df[df.labels.notnull()]

    # series_list = list()
    # print("Creating OHE for classes")
    # for idx, cls in enumerate(classes):
    #     if idx %50 == 0: print(idx)
    #     series = df['filename'].apply(lambda x: 1 if cls in x else 0)
    #     series_list.append(series)
    #     # df[str(cls)] = 1 if df['folder'].str.contains(str(cls)) else 0
    
    # df = pd.concat([df, series_list], axis=1)

    #     files = os.listdir(labeled_dir)
    #     labels = labeled_dir.split("_")
    #     for file in files:
    #         continue
            # df.loc[len(df.index)] = [os.path.join(datasetdir, labeled_dir), labels]

    
    # df.head()

    return df

# df, top_k_classes = constructImageClassDataFrame(shape = (256,256), min_class_occurence=10,class_list=['Shirt','Shorts'])
# print(df.head(100))
# print(np.shape(df))
# print(top_k_classes)
# print(np.shape(top_k_classes))
    

In [68]:
datasetdir = 'E:\\img_highres'
os.chdir(datasetdir)

from msilib.schema import Directory
from random import shuffle
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE = 8
IMG_SHAPE = (512,512)


def DataLoad(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0.1,
        rescale = 1.0/255, 
    )

    height, width = shape

    df = constructImageClassDataFrame(shape, class_list=CLASS_LIST)

    print(df.sample(n=10))
    # Modify dataframe to only take images with references to classes

    print("Num classes #")
    print(len(CLASS_LIST)) 
    # for subdir

    train_dataset = imgdatagen.flow_from_dataframe(
        dataframe = df,
        directory = datasetdir,
        x_col="filename",
        y_col="labels",
        target_size = IMG_SHAPE,
        seed = 42,
        shuffle = True,
        batch_size = BATCH_SIZE,
        class_mode="categorical",
        classes = CLASS_LIST,
        subset = 'training'
    )
    val_dataset = imgdatagen.flow_from_dataframe(
        dataframe = df,
        directory = datasetdir,
        x_col="filename",
        y_col="labels",
        target_size = IMG_SHAPE,
        seed = 42,
        shuffle = True,
        batch_size = BATCH_SIZE,
        class_mode="categorical",
        classes = CLASS_LIST,
        subset = 'validation'
    )


    return train_dataset, val_dataset, 

# train_dataset, val_dataset = DataLoad((350,350), preprocessing=preprocess_input)


In [69]:
import scipy
from keras.applications.vgg16 import VGG16, preprocess_input

# Create custom preproocessing routine


train_dataset, val_dataset = DataLoad((512,512), preprocessing=preprocess_input)

print(type(train_dataset))
print(type(val_dataset))
# print(train_dataset.shape())

X_train, y_train = next(train_dataset)

print(type(X_train))
print(y_train[0])
print(np.sum(y_train, axis=1))
print(np.sum(y_train, axis=0))
print(np.shape(X_train))
print(np.shape(y_train))
# print(y_train.shape())    

Done parsing through directories
(289212, 3)
                                 folder  \
249921           Striped_Raglan_Sweater   
275194         Two-Tone_Chunky_Cardigan   
45699         Chiffon_Halter_Maxi_Dress   
151256    Island_Tropics_Buttoned_Dress   
273785           Tuxedo-Inspired_Blazer   
187977                          NYC_Tee   
105222  EPTM._Tribal_Print_Track_Jacket   
232356   Slub_Knit_Buttoned-Back_Blouse   
196047         Paisley_Print_Maxi_Dress   
50569              Classic_Denim_Shorts   

                                                filename              labels  
249921           Striped_Raglan_Sweater/img_00000041.jpg  [Striped, Sweater]  
275194         Two-Tone_Chunky_Cardigan/img_00000017.jpg          [Cardigan]  
45699         Chiffon_Halter_Maxi_Dress/img_00000008.jpg             [Dress]  
151256    Island_Tropics_Buttoned_Dress/img_00000043.jpg             [Dress]  
273785           Tuxedo-Inspired_Blazer/img_00000133.jpg            [Blazer]  
187977 

In [72]:
y_peek = y_train[:5]

In [73]:
vgg16 = keras.applications.vgg16
conv_model = vgg16.VGG16(weights='imagenet', include_top=False)
conv_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [74]:
conv_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(512,512,3))

# flatten the output of the convolutional part: 
x = keras.layers.Flatten()(conv_model.output)
# three hidden layers}
x = keras.layers.Dense(20, activation='relu')(x)
x = keras.layers.Dense(20, activation='relu')(x)
x = keras.layers.Dense(20, activation='relu')(x)
# final softmax layer with 40 categories

predictions = keras.layers.Dense(14, activation = 'sigmoid')(x)
for layer in conv_model.layers:
    layer.trainable = False
# creating the full model:
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 512, 512, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 512, 512, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 512, 512, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 256, 256, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 256, 256, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 256, 256, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 128, 128, 128)     0   

In [75]:
# Configure run environment
from tensorflow.python.client import device_lib

device_lib.list_local_devices()
# tf.config.list_physical_devices('CPU')

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18228482594933148473
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5230841856
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8765513986292396532
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

# Categorical Cross Entropy

In [76]:
from keras.callbacks import ModelCheckpoint
import time
histories = list()
for learning_rate in [0.0001]:

    checkpoint = ModelCheckpoint("C:\\Users\mdc20\\best_model_14_class_lr@" + str(learning_rate) + "-" + str(time.time()) + ".hdf5", monitor='val_acc', verbose=1,
        save_best_only=True, mode='auto', period=1,)
# from tf.keras.losses import BinaryCrossentropy

    # cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM, from_logits=False)
    # bce = tf.keras.losses.binary_crossentropy

    # bce = BinaryCrossentropy()
    full_model.compile(loss='binary_crossentropy',
                    optimizer=keras.optimizers.Adamax(learning_rate=learning_rate, beta_1 = 0.9, beta_2=0.99),
                    metrics=['acc', tf.keras.metrics.CategoricalCrossentropy()],
                    )
    history = full_model.fit(
    train_dataset, 
    validation_data = val_dataset,
    workers=1,
    epochs=20,
    callbacks=[checkpoint]
    )
    histories.append(history)

print(type(histories))

Epoch 1/20
27719/27719 [==============================] - ETA: 0s - loss: 0.1849 - acc: 0.4879 - categorical_crossentropy: 1.8243
Epoch 1: val_acc improved from -inf to 0.54217, saving model to C:\Users\mdc20\best_model_14_class_lr@0.0001-1665971573.2646348.hdf5
27719/27719 [==============================] - 6303s 227ms/step - loss: 0.1849 - acc: 0.4879 - categorical_crossentropy: 1.8243 - val_loss: 0.1627 - val_acc: 0.5422 - val_categorical_crossentropy: 1.5722
Epoch 2/20
27719/27719 [==============================] - ETA: 0s - loss: 0.1608 - acc: 0.5601 - categorical_crossentropy: 1.5919
Epoch 2: val_acc improved from 0.54217 to 0.57156, saving model to C:\Users\mdc20\best_model_14_class_lr@0.0001-1665971573.2646348.hdf5
27719/27719 [==============================] - 3729s 135ms/step - loss: 0.1608 - acc: 0.5601 - categorical_crossentropy: 1.5919 - val_loss: 0.1522 - val_acc: 0.5716 - val_categorical_crossentropy: 1.4717
Epoch 3/20
27719/27719 [==============================] - ETA: 

KeyboardInterrupt: 

In [21]:
from plot_keras_history import show_history, plot_history
import os
os.chdir('C:\\Users\\mdc20\\')
print(os.getcwd())



plot_keras_history(history)

full_model.save('type_kernel.keras_model')

C:\Users\mdc20
Epoch 1/10
1991/1991 [==============================] - ETA: 0s - loss: 24.8112 - acc: 0.5546 - categorical_crossentropy: 24.8112
Epoch 1: acc improved from -inf to 0.55458, saving model to best_model.hdf5
1991/1991 [==============================] - 722s 362ms/step - loss: 24.8112 - acc: 0.5546 - categorical_crossentropy: 24.8112 - val_loss: 18.4811 - val_acc: 0.5966 - val_categorical_crossentropy: 18.4811
Epoch 2/10
1991/1991 [==============================] - ETA: 0s - loss: 26.5472 - acc: 0.5445 - categorical_crossentropy: 26.5472
Epoch 2: acc did not improve from 0.55458
1991/1991 [==============================] - 653s 328ms/step - loss: 26.5472 - acc: 0.5445 - categorical_crossentropy: 26.5472 - val_loss: 14.1841 - val_acc: 0.5125 - val_categorical_crossentropy: 14.1841
Epoch 3/10
1991/1991 [==============================] - ETA: 0s - loss: 34.2294 - acc: 0.5302 - categorical_crossentropy: 34.2294
Epoch 3: acc did not improve from 0.55458
1991/1991 [==============

KeyboardInterrupt: 

In [ ]:
history3 = full_model.fit(
    train_dataset, 
    validation_data = val_dataset,
    workers=1,
    epochs=20,
)

In [ ]:
print(history)

In [ ]:
# full_model.save('sparse_categorical_cross_entropy.h5')
full_model.save('39_acc.h5')

In [ ]:
from keras.models import load_model
model = load_model('cat_cross_entropy_16_classes.h5')

In [ ]:
from sklearn.metrics import confusion_matrix

x_val, y_val = next(val_dataset)
y_prediction = model.predict(x_val)
print(y_prediction[2])
print(y_val[2])
print(np.shape(y_prediction))
# preds = np.argmax(y_prediction, axis=1)
# print(preds)

k = 4
axis = 1 # Search for top k values in row
top_k_indx = np.argpartition(y_prediction, -4, axis=axis)[-4:].T
print(top_k_indx)
print(np.shape(top_k_indx))
# result = confusion_matrix(y_val, y_prediction, normalize='y_pred=')
# print(y_prediction)